In [ ]:
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns

clubs_df = pd.read_pickle('../cleaned_data/clubs')
games_df = pd.read_pickle('../cleaned_data/games.pkl')
club_games_df = pd.read_pickle('../cleaned_data/club_games.pkl')
events_df = pd.read_pickle('../cleaned_data/game_events.pkl')